In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [3]:
data_path = "C:\\Users\\Niraj Raut\\OneDrive\\Desktop\\car_simulator\\"

# Load the driving_log.csv file into a DataFrame
driving_log = pd.read_csv(data_path + 'driving_log.csv')

# Keep only the center camera images and steering angles
data = driving_log[['center', 'steering']]

In [4]:
train_data, test_data = train_test_split(data, test_size=0.2)

In [5]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(train_data,
                                                    directory=data_path,
                                                    x_col='center',
                                                    y_col='steering',
                                                    target_size=(160, 320),
                                                    batch_size=32,
                                                    class_mode='raw')

test_generator = test_datagen.flow_from_dataframe(test_data,
                                                  directory=data_path,
                                                  x_col='center',
                                                  y_col='steering',
                                                  target_size=(160, 320),
                                                  batch_size=32,
                                                  class_mode='raw')


Found 2723 validated image filenames.
Found 681 validated image filenames.


In [6]:
def create_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(160, 320, 3)))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='linear'))
    return model

In [7]:
model = create_model()

In [8]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [9]:
# Train the model
history = model.fit(train_generator,
                    steps_per_epoch=len(train_data)//32,
                    epochs=10,
                    validation_data=test_generator,
                    validation_steps=len(test_data)//32)

Epoch 1/10
85/85 [==============================] - 230s 3s/step - loss: 20.9271 - val_loss: 0.2042
Epoch 2/10
85/85 [==============================] - 180s 2s/step - loss: 0.1983 - val_loss: 0.2056
Epoch 3/10
85/85 [==============================] - 78s 900ms/step - loss: 0.2010 - val_loss: 0.2045
Epoch 4/10
85/85 [==============================] - 73s 862ms/step - loss: 0.1996 - val_loss: 0.2057
Epoch 5/10
85/85 [==============================] - 79s 925ms/step - loss: 0.2005 - val_loss: 0.2051
Epoch 6/10
85/85 [==============================] - 80s 943ms/step - loss: 0.2006 - val_loss: 0.2011
Epoch 7/10
85/85 [==============================] - 80s 941ms/step - loss: 0.2000 - val_loss: 0.2030
Epoch 8/10
85/85 [==============================] - 83s 977ms/step - loss: 0.2005 - val_loss: 0.2032
Epoch 9/10
85/85 [==============================] - 86s 1s/step - loss: 0.2011 - val_loss: 0.2068
Epoch 10/10
85/85 [==============================] - 87s 1s/step - loss: 0.2002 - val_loss: 0.204

In [10]:
# Test the model
loss = model.evaluate(test_generator, steps=len(test_data)//32)
print("Test loss: ", loss)

21/21 [==============================] - 4s 191ms/step - loss: 0.2039
Test loss:  0.2038845717906952


In [11]:
model.save('model.h5')

In [13]:
# Load the trained model
model = tf.keras.models.load_model('model.h5')

# Load and preprocess the image
img_path = "C:\\Users\\Niraj Raut\\OneDrive\\Desktop\\car_simulator\\self_driving_car_dataset_jungle\\IMG\\center_2022_04_10_12_44_27_913.jpg"

img = tf.keras.preprocessing.image.load_img(img_path, target_size=(160, 320))
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = img_array / 255.

# Make the prediction
steering_angle = model.predict(img_array)[0][0]

print("Predicted steering angle:", steering_angle)


Predicted steering angle: 0.00018913487
